# Swedish Lighthouses Challenge


Skydio is scoping out a project to aid the search and rescue operations of the Swedish Coast Guard. Their goal is to establish a network of docked drones along the coastline that provide a rapid response to emergency transmissions.

(The Skydio Dock is a cloud-connected, weatherproof home base and charging station that enables our drones to fly without a pilot on-scene.)

The customer is interested in establishing coverage from their main office in Stockholm east to the island of Vindö and then southwest to Torö.

They have permission to build a dock into any lighthouse in the region. They are interested in building a route such that a drone can hop from dock to dock from Stockholm to Torö, so that they can recover their fleet and swap drones without sending humans to the lighthouses.

**Submission** - Make a copy of this notebook and fill out the code and writeup. Feel free to download and run locally if helpful. Share the completed Google Colab link with us by setting the permission to *Anyone with the link can view*.

**Guidelines** - This challenge tests the ability to write quality code, interact with APIs, visualize data, and employ algorithms to solve problems. We score submissions by the clarity of the code, visuals, and written documentation. We do not score by the amount of time spent, so go as deep as you like, or if you are short on time then take a simpler approach to Part 2 and 3.

<img width="500px" src="https://i.imgur.com/5H88NQ3.png"/>

# Part 1: Find all of the lighthouses

Use [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap) to find known lighthouses in the region and get that data into your notebook. OSM provides APIs to query their geographic database by structure type. Plot the lighthouses on a map using [pydeck](https://deckgl.readthedocs.io/en/latest/).

Note that you are free to install python packages in colab: `!pip install pydeck`

In [1]:
!pip install overpy
!pip install pandas
!pip install pydeck
!pip install haversine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.8 MB/s eta 0:00:00


In [2]:
# Importing required modules
import overpy
import pandas as pd
import pydeck as pdk
from queue import PriorityQueue
from haversine import haversine, Unit
import numpy as np

In [3]:
# Defining some basic/fundamental quantities which will be use in planning the route.
# You can play around with different values of flight_time and flight_speed.
flight_time = 20*60 # in seconds
flight_speed = 12 # in m/s
max_flight_dist = flight_time * flight_speed
cost_of_dock = 10000

In [4]:
# Some utility functions

def create_df(columns, nodelist):
    df_1 = pd.DataFrame(columns = columns)
    for node in nodelist:
        df_1.loc[len(df_1)] = {df.columns[0]: node.id, df.columns[1]: node.name, df.columns[2]: node.lng, df.columns[3]: node.lat}
    df_1 = df_1.iloc[::-1] # Just reversing the order of rows for ease of calculations
    return df_1



# This is used in creating map, the inputs are data (longitude, latitude), desired color to display and a bool to display text
def create_layer(data, color, show_text=True):
    scatter_layer = pdk.Layer(
    'ScatterplotLayer',
    data=data,
    get_position='[longitude, latitude]',
    get_radius=1000,
    get_fill_color=color,
    pickable=True,
    )

    text_layer = pdk.Layer(
    "TextLayer",
    data=data,
    get_position=["longitude", "latitude"],
    get_text="name",
    get_size=12,
    get_color=[255, 255, 255],  # White color for text
    pickable=True,
    auto_highlight=True,
    )
    if show_text:
      return [scatter_layer, text_layer]
    else:
      return scatter_layer

# This function calculates the time to recharge a drone.
# Inputs are the present battery level:curr_batt_level and
# time taken to charge(in min):time_to_full_charge to 100%. Deafult value of time to carge is set as 45 mins
def time_to_charge(curr_batt_level, time_to_full_charge=45):
    remaining_batt_level = 100 - curr_batt_level
    return (time_to_full_charge/100) * remaining_batt_level

# This function calculates the discharge percenatge. Inputs are flying distance of the drone:flight_dist and
# the maximum distance the drone can fly:max_flight_dist
def discharge(flight_dist, max_flight_dist=max_flight_dist):
    return flight_dist * 100/max_flight_dist

# This function creates and displays map. Inputs are the layers:layer,
# viewing latitude:lat and longitude:lng
def create_map(layer, lat, lng):

    view_state = pdk.ViewState(
        latitude=lat,
        longitude=lng,
        zoom=7,
    )


    pydeck_map = pdk.Deck(
        layers=layer,
        initial_view_state=view_state,
    )

    pydeck_map.show()


In [5]:
# This code block extracts lighthouse data

# Initializing the api
api = overpy.Overpass()

# Defining an approximate bounding box where we will look for lighthouse.
# The coordinates were found manually using maps.
box = (58.7, 17.8, 59.5, 18.9)

# Creating a query to find lighthouses along the coastline.
query = f"""
node["man_made"="lighthouse"]({box[0]}, {box[1]}, {box[2]}, {box[3]});
out center;
"""

# Getting the result using api and query
result = api.query(query)

# Storing the data in a dictionary
lighthouse_data = {
    'id': [node.id for node in result.nodes],
    'name': [node.tags.get('name', 'N/A') for node in result.nodes],
    'longitude': [float(node.lon) for node in result.nodes],
    'latitude': [float(node.lat) for node in result.nodes],
}


In [6]:
# Creating a pandas dataframe. This will be used as input to create layers for the map.
df = pd.DataFrame(lighthouse_data)

# Storing the mean latitude and mean longitude. This will be passed to create_map() function.
# This will help in getting a consistent view of the map
latitude=df['latitude'].mean()
longitude=df['longitude'].mean()

In [7]:
# Displaying all the lighthouses
layer_0 = create_layer(df, [255, 0, 0], show_text = False)
create_map(layer_0, latitude, longitude)

<IPython.core.display.Javascript object>

# Part 2: Choose where to build docks

Suggest a minimal set of lighthouses at which to build a dock that would allow a drone to traverse from Stockholm to Torö, if each drone has a usable battery life of 30 minutes and travels at a ground speed of 12 meters per second. Plot the route, assuming approximate endpoints at Stockholm and Torö. Also print the total route distance.

It's okay to make reasonable assumptions here and state your thinking. You can go as straightforward or complex as you like.

It is given that the flight time is 30 mins and flight speed is 12m/s. This is under ideal conditions. In real world, the flight time will be less (different rate of battery discharge), and the flight speed will also be slower than 12m/s. Hence, I have assumed the effective flight time to be 20 mins instead of 30 mins. A star algorithm is used to find the feasible route.

In [8]:
# Defining a class called Node. Each lighthouse will be stored as a node.
# It will contain all necessary information of the lighthouse.
class Node():
    def __init__(self, self_id, name, lng, lat):
        self.id = self_id
        self.name = name
        self.lng = float(lng)
        self.lat = float(lat)
        self.neighbors = []
        self.parent_id = None

        # These vairables will be used in the A star algorithm.
        self.g = float('inf') # Cost of path from start node to this node. Initialised as infinity.
        self.h = 0.0 # Heuristic cost from this node to goal node. Initialsed as 0.
        self.f = self.g + self.h # Total cost

        self.in_open = False # Tells weather a node has been explored or needs to be explored.
        #True->has to be explored

        self.in_closed = False # Tells weather a node has been explored or needs to be explored.
        # True-> has already been explored

    # Update the value of f
    def update_f(self):
        self.f = self.g + self.h

    # Reset the node when required, example, running A star multiple times.
    def reset_node(self):
        self.parent_id = None
        self.g = float('inf')
        self.h = 0.0
        self.f = self.g + self.h
        self.in_open = False
        self.in_closed = False

In [9]:
# This function gives the haversine distance between two coordinates.
# Refer https://en.wikipedia.org/wiki/Haversine_formula to learn more about haversine distance
# This is used as the heuristic function for A star algorithm.
def get_haversine_dist(a: Node, b: Node):
    a_coord = (a.lat, a.lng)
    b_coord = (b.lat, b.lng)
    return haversine(a_coord, b_coord, unit=Unit.METERS)

In [10]:
# A star algorithm that takes in the start_node, goal_node and the node_list and returns a list called 'path' that
# contains all the nodes/lighthouse that must be followed. It also prints the name and ID of lighthouses
# from start to goal.
# Note: 'path' is in reverse order, i.e. the first element is goal and last element is start.
def A_star(start, goal, nodelist, cost_of_dock = cost_of_dock, print_path=True):

    # Initializing for A star algorithm

    # This step sets the values of every node in node list to default values
    for node in nodelist:
        node.reset_node()

    # Initalizing the first node and inserting it into priority queue
    start.g = 0
    start.update_f()
    start.in_open = True
    open_list = PriorityQueue()
    open_list.put((start.f, start))

    # Initailizing closed list. It is used to store all the explored nodes
    closed_list = []

    # Computing the path
    while goal.in_closed != True and open_list.empty() != True:

        # Removing current node from open list and inserting in closed list
        curr_node = open_list.get()[1]
        closed_list.append(curr_node)
        curr_node.in_closed = True

        # Checking all the neighboring nodes of the current node
        for node_id in curr_node.neighbors:
            for j in node_list:
                if j.id == node_id:
                    node = j
                    break
            if node.in_closed != True:
                # Computing the cost to travel to a node, updating it, and updating the open and closed list
                if node.g>curr_node.g + get_haversine_dist(curr_node, node):
                    node.g = curr_node.g + get_haversine_dist(curr_node, node) + cost_of_dock
                    node.in_open = True
                    node.parent_id = curr_node.id
                    node.h = get_haversine_dist(node, goal)
                    node.update_f()
                    open_list.put((node.f, node))

    # Checking if goal is reached or not
    if goal.in_closed == True:
        if print_path:
            print("\033[1;32mGOAL REACHED :)\033[0m")
    else:
        if print_path:
            print("\033[1;31mGOAL NOT FOUND :(\033[0m")

    current_node = goal
    path=[]
    path.append(current_node)

    # Performing the backtracking part of A star algorithm and storing the route
    while current_node.parent_id!=None:
        for node in node_list:
            if node.id == current_node.parent_id:
                current_node = node
                path.append(current_node)
                break
    # Printing the path from start to goal
    if print_path:
        for node in reversed(path):
                print(f"ID: {node.id}, Name: {node.name}")

    if print_path:
        print("\033[1;31mFINISHED\033[0m")

    return path


In [11]:
# Creating a list of nodes. Each node represents a lighthouse and stores relevant information.
# Each node also stores a list of neighboring lighthouses such that its distance from its neighbors is less than
# the maximum flight distance (refer this variable: max_flight_dist)

node_list = []

for index, row in df.iterrows():
    node_list.append(Node(row['id'], row['name'], row['longitude'], row['latitude']))

# Creating the start and goal node and appending to node_list. The ID are assigned at random,
# just one condition, there should not be two nodes with same ID.
# The coordinates were found manually using maps.
start_node = Node(100, 'Stockholm', 18.08595, 59.32842)
goal_node_1 = Node(101, 'Vindo', 18.68599, 59.35038)
goal_node_2 = Node(102, 'Toro', 17.84245, 58.82359)

node_list.append(start_node)
node_list.append(goal_node_1)
node_list.append(goal_node_2)

# Finding the neighbors for each node in node_list
for curr_node in node_list:
    for node in node_list:
        if (curr_node.id!= node.id):
            curr_coord = (curr_node.lat, curr_node.lng)
            node_coord = (node.lat, node.lng)

            if haversine(curr_coord, node_coord, unit=Unit.METERS) < max_flight_dist:
                curr_node.neighbors.append(node.id)

In [12]:
# Applying A star algorithm twice, first from Stockholm to Vindo and second time from Vindo to Toro
print(f"\033[1;36mPath 1\033[0m")
path_1 = A_star(start_node, goal_node_1, node_list)
print(f"\n\033[1;36mPath 2\033[0m")
path_2 = A_star(goal_node_1, goal_node_2, node_list)

Path 1
GOAL REACHED :)
ID: 100, Name: Stockholm
ID: 1033044045, Name: Bogesund
ID: 1033008568, Name: Valöarna
ID: 101, Name: Vindo
FINISHED

Path 2
GOAL REACHED :)
ID: 101, Name: Vindo
ID: 1033101030, Name: Franska Stenarna
ID: 1033101033, Name: Fjärdhällan
ID: 1033116566, Name: Torrbänken
ID: 1033116556, Name: Mysingeholm
ID: 1033116557, Name: Älvsnabben
ID: 1039158250, Name: Brunsviksholmen
ID: 102, Name: Toro
FINISHED


In [13]:
# Calculating the total distance of the route
dist = 0

# Calculating distance from Stockholm to Vindo
for i in range(len(path_1)-1):
    dist+=get_haversine_dist(path_1[i], path_1[i+1])

# Calculating distance from Vindo to Toro
for i in range(len(path_2)-1):
    dist+=get_haversine_dist(path_2[i], path_2[i+1])

print(f"\033[1;36mTotal distance from Stockholm to Toro is {round(dist/1000, 1)} km\033[0m")

Total distance from Stockholm to Toro is 120.6 km


In [14]:
# Creating a pandas dataframe. These will be used as input to create layers for the map.
df_1 = create_df(df.columns, path_1)
df_2 = create_df(df.columns, path_2)

In [15]:
# Displaying path_1 (Stockhol -> Vindo)
layer_1 = create_layer(df_1, [0, 255, 0])
create_map(layer_1, latitude, longitude)

<IPython.core.display.Javascript object>

In [16]:
# Displaying path_1 (Vindo -> Toro)
layer_2 = create_layer(df_2, [255, 165, 0])
create_map(layer_2, latitude, longitude)

<IPython.core.display.Javascript object>

In [17]:
# Displaying the full path Stockholm -> Vindo -> Toro
layer_3 = [create_layer(df_1, [0, 255, 255]), create_layer(df_2, [0, 255, 255])]
create_map(layer_3, latitude, longitude)

<IPython.core.display.Javascript object>

# Part 3: Get drones from Stockholm to the docks!

Let's say we installed empty docks into each of the selected lighthouses. How long would it take to fill every dock if we are dispatching the drones from Stockholm, and a drone takes 45 minutes to fully charge its battery in the dock?

You may choose to write code here at any level of depth for this problem, or you can provide a ballpark estimate with a few written sentences.

In [18]:
# Combining the two path into one and making it into a dataframe.
# This dataframe will be used to compute the time to fill all docks.
full_df = pd.concat([df_1, df_2], ignore_index=True)
full_df = full_df.drop_duplicates()

# Displaying the dataframe
full_df

,id,name,longitude,latitude
0,100,Stockholm,18.085950,59.328420
1,1033044045,Bogesund,18.304008,59.380807
2,1033008568,Valöarna,18.497744,59.432716
3,101,Vindo,18.685990,59.350380
5,1033101030,Franska Stenarna,18.665500,59.247859
6,1033101033,Fjärdhällan,18.554695,59.155542
7,1033116566,Torrbänken,18.367085,59.110389
8,1033116556,Mysingeholm,18.257957,59.002893
9,1033116557,Älvsnabben,18.186326,58.978232
10,1039158250,Brunsviksholmen,17.977776,58.918058


In [19]:
# Calculating relevant data such as flight distance, flight time, time to charge battery, etc.
# Finally summing time up to get the total time to fill all the dock.
# Assuming that drone is uncharged at Stockholm, hence needs to charge and no charging is required after reaching
# destination Toro
coords = full_df[['longitude', 'latitude']].to_numpy()
dist = []
i = 0
while i<coords.shape[0]-1:
    dist.append(haversine((coords[i][1], coords[i][0]), (coords[i+1][1], coords[i+1][0])))
    i+=1
dist.append(0)
full_df['dist_km'] = dist
full_df['flight_time_min'] = (1000 * full_df['dist_km']/flight_speed)/60
full_df['battery_level'] = (100 - (1000 * full_df['dist_km'].apply(discharge))).shift(1)
full_df.loc[0, 'battery_level']=0
full_df['time_to_charge_min'] = full_df['battery_level'].apply(time_to_charge)
full_df['total_time'] = full_df['flight_time_min'] + full_df['time_to_charge_min']

In [20]:
# This is the toal time needed to fill all docks
total_time_to_fill_all_docks = full_df['total_time'].sum()/60
print(f"\033[1;36mEstimated time for filling all docks is {round(total_time_to_fill_all_docks, 1)} hrs, when flight time is {flight_time/60} mins\033[0m")

Estimated time for filling all docks is 9.8 hrs, when flight time is 20.0 mins


# Discussion

Describe roughly the amount of time you spent on this challenge and any notes you would like to share about your approach.

I completed this challenge in three sittings of 4-5hrs each (including optional section). I took the following approach:

Coding:
1. If same task is done more than 2 time, create a function.
2. Declare all basic/fundamental variables at the start.
3. Add descriptive comments for future reference.

Approach:
1. I divided the path planning into two parts, Stockholm to Vindo and Vindo to Toro. A direct path can also be found from Stockhom to Toro, however it does not pass directly through Vindo, hence this approach was taken.
2. Found the most feasible path for each part using A star algorithm and combined them together.
3. For heuristic function, I used haversine distance between two locations.
4. I added a cost for building docks into the total cost (g). This helped in reducing the number of docks required to be installed. You can change the cost_of_dock variable to create and test different scenarios.
5. Effective flight time was assumed to be 20 mins. You can change the value of 'flight_time' and see its effect.

I have tried to make the code flexible wrt the most important/fundamental variables. These are declared at the begining and you can play with the values and see their effects.


# Stretch goal (optional)

Analyze and visualize any other interesting aspect of this proposal. What is some problem we might run into or information the customer could want to know about using this network of docked drones for search and rescue?

I have created and displayed a table which contains information such as distance between consecutive lighthouses, time required, time to recharge. These may be usefull for further execution of this project.

In [21]:
# Displaying the datafarme
full_df

,id,name,longitude,latitude,dist_km,flight_time_min,battery_level,time_to_charge_min,total_time
0,100,Stockholm,18.085950,59.328420,13.663231,18.976710,0.000000,45.000000,63.976710
1,1033044045,Bogesund,18.304008,59.380807,12.390367,17.208843,5.116448,42.697598,59.906442
2,1033008568,Valöarna,18.497744,59.432716,14.050317,19.514329,13.955783,38.719898,58.234227
3,101,Vindo,18.685990,59.350380,11.459060,15.915361,2.428353,43.907241,59.822602
5,1033101030,Franska Stenarna,18.665500,59.247859,12.048734,16.734353,20.423196,35.809562,52.543915
6,1033101033,Fjärdhällan,18.554695,59.155542,11.821983,16.419421,16.328235,37.652294,54.071715
7,1033116566,Torrbänken,18.367085,59.110389,13.483495,18.727076,17.902894,36.943698,55.670774
8,1033116556,Mysingeholm,18.257957,59.002893,4.935359,6.854665,6.364618,42.135922,48.990587
9,1033116557,Älvsnabben,18.186326,58.978232,13.705768,19.035789,65.726676,15.422996,34.458785
10,1039158250,Brunsviksholmen,17.977776,58.918058,13.071236,18.154494,4.821055,42.830525,60.985019


We can also see the effect taking into account the cost of installing docks and/or changing the flight time. Below is a small function that will help visualize the effect.

To see the effect of flight_time, change 'flight_time' and rerun the code.

In [22]:
def cost_vs_dock(start, goal1, goal2, nodelist, cost, print_path=False):
    start_node = start
    goal_node_1 = goal1
    goal_node_2 = goal2
    node_list = nodelist
    # Applying A star algorithm twice, first from Stockholm to Vindo and second time from Vindo to Toro
    path_3 = A_star(start_node, goal_node_1, node_list, cost, print_path)
    path_4 = A_star(goal_node_1, goal_node_2, node_list, cost, print_path)

    # Creating pandas dataframe from the returned path (from Stockholm to Vindo)
    df_3 = create_df(df.columns, path_3)

    # Creating pandas dataframe from the returned path (from Vindo to Toro)
    df_4 = create_df(df.columns, path_4)

    # Displaying the full path Stockholm -> Vindo -> Toro
    layer_5 = [create_layer(df_3, [0, 255, 255], False), create_layer(df_4, [0, 255, 255], False)]
    create_map(layer_5, latitude, longitude)
    print(f"\033[1;36mTotal docks required is {len(path_3) + len(path_4) - 1}, when flight time is {flight_time/60} mins and cost of installing dock in {cost}\033[0m")


In [23]:
# When the cost of installing dock is set to 0
dock_cost = 0
cost_vs_dock(start_node, goal_node_1, goal_node_2, node_list, dock_cost)

<IPython.core.display.Javascript object>

Total docks required is 13, when flight time is 20.0 mins and cost of installing dock in 0


In [24]:
# When the cost of installing dock is set to 10
dock_cost = 10
cost_vs_dock(start_node, goal_node_1, goal_node_2, node_list, dock_cost)

<IPython.core.display.Javascript object>

Total docks required is 12, when flight time is 20.0 mins and cost of installing dock in 10


In [25]:
# When the cost of installing dock is set to 100
dock_cost = 100
cost_vs_dock(start_node, goal_node_1, goal_node_2, node_list, dock_cost)

<IPython.core.display.Javascript object>

Total docks required is 11, when flight time is 20.0 mins and cost of installing dock in 100


In [26]:
# When the cost of installing dock is set to 1000
dock_cost = 1000
cost_vs_dock(start_node, goal_node_1, goal_node_2, node_list, dock_cost)

<IPython.core.display.Javascript object>

Total docks required is 11, when flight time is 20.0 mins and cost of installing dock in 1000


We notice that as the cost is increased from 0 to 100, the number of required docks reduces from 13 to 11. Any further increase in the cost of installing dock has no effect on the number of required docks (see when the cost is set to 1000). Hence we can be confident that atleast 11 docks are required.

NOTE: These results are under the assumption that effective flight time is 20 mins. To see the effect of flight time on the route, you can change the value of the variable 'flight_time' and re-run the code.

Open to suggestions and happy to discuss more. You can contact me at sourojit.saha@gmail.com